# Tutorial on manipulating SLSTR L1 RBT product

In [ ]:
from pathlib import Path
import xarray as xr
from eopf import EOProduct
from eopf.common.constants import OpeningMode


In [ ]:
SAMPLE_PATH = Path("/mount/internal/work-st/projects/cs-412/2078-dpr/Samples/Products/Zarr_Beta")
product = SAMPLE_PATH / "S3SLSRBT_20230315T095847_0179_B150_S106.zarr/"

### Open the product using EOPF API and visualize the tree structure
Note: there are still warning messages due to problems of dependencies in the CPM package

In [ ]:
prod=EOProduct("prod",url=str(product)).open(mode=OpeningMode.OPEN)
prod


### Opening gridded measurement data data with xarray

In [ ]:
group = "measurements/inadir"
# Open with xarray zarr engine
ds=xr.open_dataset(SAMPLE_PATH / product / group, engine="zarr",chunks={})
# Open with custom engine="eop"
ds=xr.open_dataset(SAMPLE_PATH / product, group=group, engine="eop")
ds


The underlying data is dask.array

In [ ]:
ds.s7_bt_in.data

In [ ]:
import matplotlib.pyplot as plt
bt=ds.s7_bt_in
plt.figure(figsize=(8, 8))
ax = plt.axes()
bt.plot.pcolormesh(
    ax=ax,
    x="longitude", y="latitude", add_colorbar=False
)

### Use the confidence flag to plot land pixels

In [ ]:
group = "quality/inadir"
flag = xr.open_dataset(SAMPLE_PATH / product ,group=group,engine="eop")
meanings=flag.confidence_in.attrs["flag_meanings"].split()
land_bit = meanings.index("land")
land = (flag.confidence_in & 2**land_bit == 2**land_bit)
land.plot()

### Open the meteo data and interpolate the temperature profile at P=980hPa

In [ ]:
group = "conditions/meteo"
ds=xr.open_dataset(SAMPLE_PATH / product / group, engine="zarr")
ds.temperature_profile_tp.interp(p_atmos=980).plot()